# Data Preprocessing

Student names: FIRST_STUDENT, SECOND_STUDENT, THIRD_STUDENT, FORTH_STUDENT

Team number: YOUR_REGISTERED_TEAM_NUMBER_ON_CANVAS

The following code is used to preprocess the datasets into files that are used for the data story.

In [ ]:
import pandas as pd
######################################################
# Converting wide-format datasets to long-format ones#
######################################################

def convert_wide_to_long(file_path, skiprows=4, id_vars=None, var_name="Year", value_name="Value"):
    # Load the CSV file
    df = pd.read_csv(file_path, skiprows=skiprows)
    
    # Automatically determine id_vars if not provided
    if id_vars is None:
        id_vars = df.columns[:4]  # First 4 columns are IDs
    
    # Melt the DataFrame
    df_long = pd.melt(df, id_vars=id_vars, var_name=var_name, value_name=value_name)
    
    # Optionally drop rows with missing values in the value column
    # df_long.dropna(subset=[value_name], inplace=True)

    # Display the result
    print(df_long.head())
    
    return df_long

# Convert economy csv to long format
file_path1 = "World bank wide format/API_3_DS2_en_csv_v2_390106.csv"
df_long1 = convert_wide_to_long(file_path1)

# convert GINI csv to long format
filepath2 = "World Bank Gini wide/API_SI.POV.GINI_DS2_en_csv_v2_2566.csv"
df_long2 = convert_wide_to_long(filepath2)

#save to path
output_csv_path1 = "world bank long format2.csv"
output_csv_path2 = "gini long2.csv"

#df_long1.to_csv(output_csv_path1, index=False)
#df_long2.to_csv(output_csv_path2, index=False)

###############################################################################
# EU dataset pivot so that the different types of crime get their own columns #
###############################################################################

# Load the dataset
eu_df = pd.read_csv("europe crime.csv")

# Pivot the data
df_pivot = eu_df.pivot_table(
    index=["Geopolitical entity (reporting)", "TIME_PERIOD"],
    columns="International classification of crime for statistical purposes (ICCS)",
    values="OBS_VALUE"
).reset_index()

# save to new file
df_pivot.to_csv("europe crime testingasl;dkfja;sdlfjasdf;l.csv")

# Display the first few rows
print(df_pivot.head())
